# CODE FOR AUTOMATION OF DATA COLLECTION Germanium detector 
## LBNL lab 70-0141
## By Damien Bowen , updated: Lisa Schlueter September 2024

### Initialization (packages, connecting to instrumentation)

In [2]:
## packages
import shutil
import pyvisa
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import struct 
import os
from datetime import datetime
import h5py

##### Electronics have had their ip address configured as given below

In [11]:
## initializing resource manager for the electronics
rm = pyvisa.ResourceManager('@py')

<ResourceManager(<PyVisaLibrary('py')>)>

In [19]:
## connecting to the oscilloscope (essential for both detector tests and bench tests)
scope = rm.open_resource('TCPIP0::169.254.7.109')
print(scope.query('*IDN?')) ## prints make and model of oscilloscope to verify connection

TEKTRONIX,MSO44B,SGVJ010162,CF:91.1CT FV:2.6.38.1348



In [5]:
scope.query(':TIM:SCAL?')

NameError: name 'scope' is not defined

# Definition of functions, categorized as best as possible below

### For interacting with the electronics:

In [20]:
## gets measurement from any of the measurement channels on oscilloscope
def get_measurement(measurement_channel):
    # Query the value of the specified measurement channel
    measurement = scope.query(f':MEASU:MEAS{measurement_channel}:VALue?')
    return float(measurement)

In [21]:
## make sure to adjust which channel you want to use for the trigger
def set_trigger(trigger_level):
    # Construct the SCPI command to set the trigger source
    scope.write('TRIGger:A:EDGE:SOURce CH4') ## makes sure the trigger is set to channel 4
        
    # Construct the SCPI command to set the trigger level
    scope.write(f'TRIGger:A:LEVel:CH4 {trigger_level}') ## set trigger as low as possible 

In [22]:
# getting waveform data from oscilloscope for a given channel
def get_waveform_data(channel):
    # Select the channel
    scope.write(f'DATA:SOURCE CH{channel}')
    scope.write(f'DATA:WIDTH 2')
    
    raw_data = scope.query_binary_values('CURVE?', datatype = 'b', container=np.array)

# Horizontal scale settings to reconstruct time data
    x_increment = float(scope.query('WFMPRE:XINCR?'))
    x_zero = float(scope.query('WFMPRE:XZERO?'))
    x_offset = float(scope.query('WFMPRE:PT_OFF?'))
    # print(x_offset, x_zero)

# Get vertical scale settings to reconstruct voltage data
    y_multiplier = float(scope.query('WFMPRE:YMULT?'))
    y_offset = float(scope.query('WFMPRE:YOFF?'))
    y_zero = float(scope.query('WFMPRE:YZERO?'))

# Convert raw data to numpy array
    data = np.frombuffer(raw_data, dtype=np.int16)  # Adjust dtype if needed

# Reconstruct time and voltage data
    time_data = x_zero + (np.arange(len(data)) - x_offset) * x_increment
    voltage_data = y_zero + (data - y_offset)*y_multiplier

    return time_data, voltage_data

In [23]:
## corrects the vertical scaling of the oscilloscope based on the peak voltage. 
## this function is used in combination with get_measurement() to get the peak voltage through the device 
## being tested.
def correct_y_scaling(wave_amplitude):

    ## takes in amplitude of the ASIC (a float) and adjusts y divisions of oscilloscope as required
    if 1.6 < wave_amplitude <= 4:
        set_trigger(0.1)
        set_vert_scale(1, 0.5)
        set_vert_scale(4, 0.5)
    elif 0.8 <= wave_amplitude <= 1.6:
        set_vert_scale(1, 0.2)  # 200 mV/div for both ASIC readout, pulser
        set_vert_scale(4, 0.2)
    elif 0.4 <= wave_amplitude < 0.8:
        set_vert_scale(1, 0.1)  # 100 mV/div for ASIC, still 200 for pulser
    elif 0.18 <= wave_amplitude < 0.4:
        set_trigger(0.05)
        set_vert_scale(1, 0.05)  # 50 mV/div on ASIC readout
        set_vert_scale(4, 0.1)   # 100 mV/div for pulser
    elif 0.08 <= wave_amplitude < 0.18:
        set_trigger(0.05)
        set_vert_scale(1, 0.02)  # 20 mV/div
        set_vert_scale(4, 0.05)  # 50 mV/div for pulser
    elif 0.04 <= wave_amplitude < 0.08:
        set_trigger(0.02)
        set_vert_scale(1, 0.01)  # 10 mV/div
        set_vert_scale(4, 0.02)  # 20 mV/div for pulser
    elif 0.015 <= wave_amplitude < 0.04:
        set_trigger(0.01)
        set_vert_scale(1, 0.005)  # 5 mV / div
        set_vert_scale(4, 0.01)   # 10 mV/div for pulser
    elif .008 <= wave_amplitude < 0.015:
        set_trigger(0.004)
        set_vert_scale(1, 0.002)
    elif 0.0 <= wave_amplitude < .008:
        set_vert_scale(1, 0.001)


# Saving data to different file types. Currently, .csv and .hdf5 are available (.hdf5 is significantly faster)

##### code for creation of folders:

In [24]:
## creates folder for .csv files to be inserted into. The folder is automatically created in this directory
## in the folder I have called ./Data.
## GIVE THE FOLDER A NAME IN THE ALGORITHMS
def create_folder(base_dir='./Data', name = None, run_type = None, n_runs = None, subfolder = None):
    # If a subfolder is specified, use it directly without generating a new folder name
    if subfolder:
        folder_path = os.path.join(base_dir, subfolder)
    else:
        # Generate a folder name if 'name' is not provided and 'subfolder' is not specified
        if name is None:
            name = f'ASIC_data_{int(time.time())}'
        else:
            current_date = datetime.datetime.now().strftime('%Y%m%d')
            name = f'{name}_{current_date}'
        folder_path = os.path.join(base_dir, name)
    
    # Create the folder (or subfolder) path
    os.makedirs(folder_path, exist_ok=True)
    # print(f'Folder created: {folder_path}')
    
    ## create a .csv file with important metadata for main data collection folders
    if not subfolder:
        info_filename = 'run_info.csv'
        info_filepath = os.path.join(folder_path, info_filename)
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')
        current_time = datetime.datetime.now().strftime('%H:%M:%S')

        with open(info_filepath, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Oscilloscope: TEKTRONIX MSO44B', 'Pulser: TELEDYNE T3AFG500'])
            csvwriter.writerow([f'Date: {current_date}', f'Time: {current_time}'])
            csvwriter.writerow(['Time units: microseconds', 'Voltage units: Volts'])


            if run_type:
                csvwriter.writerow([f'Run type: {run_type}'])

            if n_runs:
                csvwriter.writerow([f'# of iterations at each voltage: {n_runs}'])
    
    return folder_path

### save data to .hdf5 file:

In [27]:
## data to write a waveform into an hdf5 file
def write_waveform_to_hdf5(hdf_file, group_name, time_data, voltage_data, run_num):
    """
    Writes waveform data to HDF5 file under the specified group.

    Parameters:
    hdf_file (h5py.File): The open HDF5 file object.
    group_name (str): The name of the group (e.g., voltage level).
    time_data (list or np.array): The time data points.
    voltage_data (list or np.array): The corresponding voltage data points.
    run_num (int): The current run number to name the dataset.
    """
    # Combine the time and voltage data into a 2D array
    waveform_data = np.vstack((time_data, voltage_data)).T
    
    # Create a dataset for the waveform in the specified group
    dataset_name = f"waveform_{run_num:05d}"
    
    # Write the dataset with compression
    hdf_file[group_name].create_dataset(dataset_name, data=waveform_data, compression="gzip")
    
    """Optional print statement for debugging and tracking."""
    # print(f"Waveform {run_num + 1} saved to group {group_name}.")

In [28]:
## Function to automatically upload data to google drive

def save_hdf5_to_drive(hdf_file_path):
    
    ## constructing the path to the file you want to upload to the google drive
    source_file_path = hdf_file_path
    
    ## constructing path to the google drive
    google_drive_path = google_drive_path = r"/Users/ilvuoto/Library/CloudStorage/GoogleDrive-dvbowen@lbl.gov/My Drive/ASIC_data_drive"

    ## moving the source file to the google drive
    shutil.move(source_file_path, google_drive_path)
    
    print(f"Moved data file to {google_drive_path}.")
    

In [29]:
### function to write important metadata into each file

def write_hdf5_metadata_osci(hdf_file, channel):    
    ### METADATA
    # hdf_file.attrs['DEVICE_NAME'] = 'LBNL GFET distributed CSA'
    # hdf_file.attrs['DEVICE_VOLTAGE'] = '1.0 V'
    # hdf_file.attrs['V_MID'] = '0.8 V'
    # hdf_file.attrs['POWER_SUPPLY_PARAMETERS'] = ['5V, 3mA and -5V, 3mA']
    # hdf_file.attrs['SCOPE_PARAMETERS'] = ['AC termination, 75 Ohms']
    # hdf_file.attrs['HPGe_LBNL_PPC_BIAS_VOLTAGE'] = ['2 kV']
    scope_group = hdf_file.create_group('oscilloscope')
    
    scope.write(f':CHANnel{channel}:IMPedance?')
    input_impedance = scope.read().strip()

    scope.write(f':CHANnel{channel}:COUPling?')
    coupling_mode = scope.read().strip()
    
     # Add to file 
    scope_group.create_dataset('input_impedance', data=input_impedance)
    scope_group.create_dataset('coupling_mode', data=coupling_mode)
    # hdf_file.attrs['INPUT_IMPEDANCE'] = input_impedance
    # hdf_file.attrs['COUPLING_MODE'] = coupling_mode
    
    # if run_type:
    #     hdf_file.attrs['RUN_TYPE'] = run_type

In [ ]:
def write_hdf5_metadata_detector(hdf_file: h5py.File, bias_voltage_V: float): 
    det_group = hdf_file.create_group('detector')
    
    bias_voltage_dataset = det_group.create_dataset('bias_voltage', data=bias_voltage_V)
    bias_voltage_dataset.attrs['unit'] = 'V'
    

def write_hdf5_metadata_csa(hdf_file, csa_name:str, csa_bias_V: float, resistor_ohm: float, resistor_type: str, csa_info: str = ""): 
    csa_group = hdf_file.create_group('CSA')
    csa_group.attrs['type'] = csa_name
    if csa_aux != "":
        csa_group.attrs['info'] = csa_info

    resistor_dataset = csa_group.create_dataset('resistor', data=resistor_ohm)
    resistor_dataset.attrs['unit'] = 'Ohm'
    resistor_dataset.attrs['type'] = resistor_type

    bias_dataset = csa_group.create_dataset('bias', data=csa_bias_V)
    bias_dataset.attrs['unit'] = 'V'
   

    

## DATA COLLECTION FUNCTIONS

In [33]:
## FOR USE WITH DETECTOR

# PARAMETERS:
# max_events: number of events saved to .csv file before algorithm terminates
def data_scan(max_events = 1000):
    
    folder_path = create_folder(base_dir = "./Data", name = "ASIC_detector_run_test3")
    count = 0
    print("Scanning begins..")
    
    while True:
    # Poll the oscilloscope's trigger status
        scope.write('TRIGger:STATE?')
        trigger_status = scope.read().strip()
    
    # Check if the trigger status indicates an event
        if trigger_status == 'TRIGGER':  # This indicates an event occurred
            print("Event detected!")
        
            time_wave, voltage_wave               = get_waveform_data(1) ## gets ASIC waveform data out of channel 1
            time_wave_pulser, voltage_wave_pulser = get_waveform_data(2) ## gets pulser voltages 
            save_to_csv_detector(path = folder_path, time_data = time_wave, device_voltage_data = voltage_wave, pulser_voltage_data = voltage_wave_pulser, counter = count)
        
            ## counter to keep track of how many events have been saved
            count += 1
        
            if count >= max_events:
                break

##### hdf5 file method:

In [40]:
## FOR USE WITH DETECTOR

# PARAMETERS same as .csv version.
def data_scan_hdf5(max_events=1000):
        
    # Open HDF5 file for storing the waveforms
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    hdf5_file_path = f'./Data/ASIC_detector_run_{timestamp}.hdf5'  # You can modify the name or path as needed
    hdf_file = h5py.File(hdf5_file_path, 'a')    # Open HDF5 in append mode
    
    write_hdf5_metadata(hdf_file, run_type = 'Detector test')
    
    run_title = f"ASIC_detector_run_{timestamp}"
    run_group = hdf_file.require_group(run_title) ## creating a group for this run so hierarchy is the same
    
    group_name = run_group.name
    
    count = 0
    print("Scanning begins..")
    
    while True:
        # Poll the oscilloscope's trigger status
        scope.write('TRIGger:STATE?')
        trigger_status = scope.read().strip()
    
        # Check if the trigger status indicates an event
        if trigger_status == 'TRIGGER':  # This indicates an event occurred
            # print("Event detected!")

            # Get the waveform data
            time_wave, voltage_wave = get_waveform_data(1)  # ASIC waveform data out of channel 1
            # time_wave_pulser, voltage_wave_pulser = get_waveform_data(2)  # Pulser voltages
            
            if count == 0:
                waveforms_group = hdf_file.require_group('waveforms')
                time_dataset =  waveforms_group.create_dataset('timestep', data= time_wave, maxshape=(None,), chunks=True)'
                time_dataset.attrs['unit'] = 's'
                # save time data to the HDF5 file -> only ONCE!
                #  Create a group for the ASIC data
                #  hdf_file.create_group(group_name)
            count += 1
            # if count % 100 == 0:  
                # print(f"Saving event {count-100} - {count} to HDF5 file.")  
            # # Save waveform to HDF5 file
            # write_waveform_to_hdf5(hdf_file, group_name, time_wave, voltage_wave, count)
        
            # Increment counter for tracking how many events have been saved
            
        
            if count >= max_events:
                voltage_dataset =  waveforms_group.create_dataset('voltage', data= voltage_wvf, maxshape=(None,), chunks=True)'
                voltage_dataset.attrs['unit'] = 'V'
                break
    
    save_hdf5_to_drive(hdf5_file_path)

# ACTUAL DATA COLLECTION

### initializing machinery:

In [ ]:
## pulser parameters
rise_time = 10e-9 
fall_time = 10e-9  ## 10 ns rise and fall time
pulse_width = .1   ## 100 ms pulse width
pulser_delay = 0.0 ## no delay

set_horizontal_scale(100e-6) ## sets time axis of oscilloscope to 40 microseconds


pulser.write(f':PULS:TRAN:LEAD {10e-9}') 
pulser.write(f':PULS:TRAN:TRA {10e-9}')  ## rise and fall time to 10 nanos
pulser.write(f':PULS:WIDth {.1}')

pulser.write(f':PULS:DEL {0.0}') ## 0 delay 

In [ ]:
## Run the configuration function for power supply
configure_power_supply()

In [ ]:
## INITIALIZING SCOPE
set_trigger(.1)
pulser_v(3.5)
set_vert_scale(4, .5)
set_vert_scale(1, .5)


## STANDARDIZE SCOPE OFFSET SO ITS IDENTICAL ACROSS RUNS
scope.write('CH1:POS -4.00')
scope.write('CH4:POS -4.00')


## SET ACQUISITION MODE TO HIGH RESOLUTION
scope.write(':ACQ:MODE HIRES')

### .hdf5 method: detector test

In [41]:
data_scan_hdf5(max_events = 1000)

Scanning begins..


KeyboardInterrupt: 

In [46]:
voltage_wvf  = []
counter = 3

for i in range(100):
     voltage_wvf.append(np.random.rand(10))

hdf_file = h5py.File('text.h5', 'a')   
# hdf_file.require_group('signal')
hdf_file['signal'].create_dataset(f"wvf{counter}", data=voltage_wvf, compression="gzip")
counter += 1
# save time data to the HDF5 file -> only ONCE!
#  Create a group for the ASIC data
#  hdf_file.create_group(group_name)

# print(f"Saving event {count-100} - {count} to HDF5 file.")  
# # Save waveform to HDF5 file
# write_waveform_to_hdf5(hdf_file, group_name, time_wave, voltage_wave, count)
        

In [47]:
def print_hdf5_structure(file_name):
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")

    with h5py.File(file_name, 'r') as hdf_file:
        hdf_file.visititems(print_attrs)
print_hdf5_structure('text.h5')

signal
signal/wvf1
signal/wvf2
signal/wvf3
signal/wvf{counter}
